In [19]:
import warnings
import sys
from pyscf import data

warnings.filterwarnings("ignore")

In [20]:
import numpy as np
import pandas as pd
import h5py
from collections import OrderedDict
from atom_rdf_pack.atom_rdf import get_rmsd_error, rad

In [3]:
atom_config = ["B+1", "B+3", "C+2", "C+4", "N+3", "N+5", "O+4", "O+6", "F+5", "F+7", "Ne+0", "Ne+6", "Ne+8"]
atom_config_with_be = atom_config + ["Be+0"]

In [4]:
atom_cfg = [(l, l.split("+")[0], int(l.split("+")[1])) for l in atom_config]
atom_cfg_with_be = [(l, l.split("+")[0], int(l.split("+")[1])) for l in atom_config_with_be]

In [5]:
def h5group_to_dict(grp):
    dct = {}
    for key, val in grp.items():
        if isinstance(val, h5py.Dataset):
            dct[key] = val[()]
        else:
            dct[key] = h5group_to_dict(val)
    return dct

In [6]:
def get_rdf_error(dct1, dct2):
    return pd.Series({key: get_rmsd_error(dct1[key], dct2[key], scale=key) for key in ["RHO", "GRD", "LR"]}) / data.nist.BOHR

## Read RDF Files

In [7]:
%%time
with h5py.File("rdf.h5", "r") as f:
    rdf_dict = h5group_to_dict(f)

CPU times: user 300 ms, sys: 420 ms, total: 720 ms
Wall time: 1.51 s


In [8]:
basis_list = [
    "aug-cc-pwcvdz", "aug-cc-pwcvtz", "aug-cc-pwcvqz", "aug-cc-pwcv5z",
    "aug-cc-pcvdz", "aug-cc-pcvtz", "aug-cc-pcvqz", "aug-cc-pcv5z",
    "aug-cc-pvdz", "aug-cc-pvtz", "aug-cc-pvqz", "aug-cc-pv5z",
    "cc-pvdz", "cc-pvtz", "cc-pvqz", "cc-pv5z",
    "def2-svpd", "def2-tzvpd", "def2-qzvpd",
    "aug-pc-1", "aug-pc-2", "aug-pc-3", "aug-pc-4",
    "aug-pcX-1", "aug-pcX-2", "aug-pcX-3", "aug-pcX-4"]

## Error of HF/MP2

In [9]:
t = {}
for atom in atom_config_with_be:
    t[atom] = get_rdf_error(rdf_dict["MP2"]["aug-cc-pwcv5z"][atom], rdf_dict["CCSD"]["aug-cc-pwcv5z"][atom])
pd.DataFrame(t)

,B+1,B+3,C+2,C+4,N+3,N+5,O+4,O+6,F+5,F+7,Ne+0,Ne+6,Ne+8,Be+0
RHO,0.822068,0.009760,0.942920,0.006070,1.068673,0.004094,1.190525,0.002920,1.305503,0.002172,0.747197,1.413852,0.001669,0.744851
GRD,0.309928,0.010204,0.493397,0.007735,0.709245,0.006170,0.952849,0.005084,1.221043,0.004293,0.282969,1.511796,0.003691,0.166261
LR,0.100475,0.004738,0.201226,0.004282,0.347781,0.003976,0.545597,0.003745,0.799583,0.003561,0.081245,1.114343,0.003406,0.039263


In [10]:
pd.DataFrame(t).T.mean()

RHO    0.590162
GRD    0.406048
LR     0.232373
dtype: float64

In [11]:
pd.DataFrame(t).T.max()

RHO    1.413852
GRD    1.511796
LR     1.114343
dtype: float64

In [12]:
t = {}
for atom in atom_config_with_be:
    t[atom] = get_rdf_error(rdf_dict["HF"]["aug-cc-pwcv5z"][atom], rdf_dict["CCSD"]["aug-cc-pwcv5z"][atom])
pd.DataFrame(t)

,B+1,B+3,C+2,C+4,N+3,N+5,O+4,O+6,F+5,F+7,Ne+0,Ne+6,Ne+8,Be+0
RHO,1.657700,0.043472,1.654443,0.032144,1.725015,0.025017,1.827710,0.020202,1.942326,0.016717,2.526159,2.060685,0.014119,1.858608
GRD,0.483349,0.058818,0.712446,0.051790,0.990533,0.046818,1.309969,0.042986,1.664883,0.039907,1.191905,2.052249,0.037356,0.314682
LR,0.123419,0.047613,0.252768,0.050796,0.444576,0.053790,0.705611,0.056501,1.042189,0.059035,0.408708,1.460903,0.061385,0.053297


In [13]:
pd.DataFrame(t).T.mean()

RHO    1.100308
GRD    0.642692
LR     0.344328
dtype: float64

In [14]:
pd.DataFrame(t).T.max()

RHO    2.526159
GRD    2.052249
LR     1.460903
dtype: float64

## Summarized table of basis error

In [15]:
basdep_dct = {}
for atom in atom_config:
    basdep_dct[atom] = {}
    for basis in basis_list:
        err = get_rdf_error(rdf_dict["CCSD"][basis][atom], rdf_dict["CCSD"]["aug-cc-pwcv5z"][atom])
        for task in ["RHO", "GRD", "LR"]:
            basdep_dct[atom][(basis, task)] = err[task]

In [16]:
res_mean = pd.DataFrame.from_dict(basdep_dct).T.mean()
res_mean = pd.DataFrame({basis: res_mean[basis] for basis in basis_list}).T

In [17]:
res_Ne8 = pd.DataFrame.from_dict(basdep_dct)["Ne+8"]
res_Ne8 = pd.DataFrame({basis: res_Ne8[basis] for basis in basis_list}).T

In [18]:
res_mean

,RHO,GRD,LR
aug-cc-pwcvdz,1.114948,1.301351,7.713953
aug-cc-pwcvtz,0.449337,0.798273,6.721336
aug-cc-pwcvqz,0.155297,0.253384,3.581307
aug-cc-pwcv5z,0.000000,0.000000,0.000000
aug-cc-pcvdz,1.410593,1.475342,7.725743
aug-cc-pcvtz,0.493062,0.805136,6.718991
aug-cc-pcvqz,0.156169,0.254968,3.580981
aug-cc-pcv5z,0.004015,0.005277,0.007591
aug-cc-pvdz,1.880921,2.499310,7.896936
aug-cc-pvtz,0.677586,0.903439,6.739854
